#1. Install packages

In [10]:
!pip install mat4py

#2.Import libraries

In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import os
import random
import mat4py 
import scipy.io


#3.Function to crop image and its mat file

In [ ]:
def crop_img(root, img_file, mat_file, n):

  # load mat file 
  mat = mat4py.loadmat(root+'ground_truth/'+mat_file)

  
  # read the image 
  img = cv2.imread(root+'images/'+img_file)

  # width and height of the image
  height, width, channels = img.shape
  
  # 'origin' is the top left point in the image, and 'end' bottom right point in the image
  x2 = random.randint(int(width/4), width)
  y2 = random.randint(int(height/4), height)

  x1 = int(x2 - int(width/4))
  y1 = int(y2 - int(height/4))
  origin = (x1, y1)
  end = (x2, y2)

  # make array of zeros with (width x height) of the image
  mat_arr = np.zeros((width, height), dtype=int)
  # assign 1 to only pixle mentioned in mat file
  for i in range(len(mat['image_info']['location'])):
    mat_arr[mat['image_info']['location'][i][0] - 1 ,mat['image_info']['location'][i][1] - 1 ] = 1
  
  # crop the image
  crop_img = img[origin[1]:end[1], origin[0]:end[0]]
  
  # crop the mat array
  crop_mat = mat_arr[origin[0]:end[0], origin[1]:end[1]]

  # only return the pixels with value = 1
  final_arr = np.where(crop_mat == 1)
  
  # if there is no person in the image, then return false 
  if len(final_arr[0]) < 1:
    return False

  # copy the dataframe and drop the portion where it cropped
  new_mat = mat['image_info']['location'][0:len(final_arr[0])]

  # mapping between the mat array and the original dimensions
  for i in range(len(new_mat)):
    new_mat[i][0] = final_arr[0][i]
    new_mat[i][1] = final_arr[1][i]
    
  mat_file = mat_file.replace('.mat', '')

  # save the mat file
  df_dict ={"image_info":np.array({"location": new_mat ,"number":len(new_mat)})}
  scipy.io.savemat(root+'ground_truth/'+mat_file+'_c'+str(n)+'.mat', df_dict, oned_as='column', appendmat = False)

  # save the new image
  path = root+img_file
  full_path, ext = os.path.splitext(path)
  img_file = full_path.replace(root, '')
  cv2.imwrite(root+'images/'+img_file+'_c'+str(n)+ext, crop_img)
  
  return True

#4.Function to mirror an image and its mat file

In [ ]:
def mirror_img(root, img_file, mat_file):
  
  # load mat file 
  mat = mat4py.loadmat(root+'ground_truth/'+mat_file)

  # load image
  img = cv2.imread(root+'images/'+img_file)
  
  # width and height of the image
  height, width, channels = img.shape

  # flip the image horizontally
  flipHorizontal = cv2.flip(img, 1)

  # get the x's, y's coordinates
  new_mat = mat['image_info']['location']

  # loop over coordinates and change the x coordinate
  for i in range(len(new_mat)):
    new_mat[i][0] = width - new_mat[i][0] - 1
  

  mat_file = mat_file.replace('.mat', '')

  # save the mat file
  df_dict ={"image_info":np.array({"location": new_mat ,"number":len(new_mat)})}
  scipy.io.savemat(root+'ground_truth/'+mat_file+'_mirror'+'.mat', df_dict, oned_as='column', appendmat = False)


  # if the extension of the image file is .jfif convert it to jpg
  if img_file.endswith('.jfif'):
    img_file = img_file.replace('jfif', 'jpg')
  

  # save the new image
  path = root+img_file
  full_path, ext = os.path.splitext(path)
  img_file = full_path.replace(root, '')
  cv2.imwrite(root+'images/'+img_file+'_mirror'+ext, flipHorizontal)


#5. Function to applay augmentation

In [ ]:
def data_augmentation(folder_path, crop_func = None, n_crop = 1, mirror_func = None):

  # root path
  root_path = folder_path

  # create an empty list to save all mat files names
  mat_files = []

  # create an empty list to save all images files names
  img_files = []

  # loop over root path, and get directories and files
  for root, dirs, files in os.walk(root_path):
    # if the folder has at least one file
    if len(files) != 0:
      # if files are images
      if files[0].endswith('.jpg') or files[0].endswith('.jpeg') or files[0].endswith('.JPG') or files[0].endswith('.png') or files[0].endswith('.jfif'):
        img_files = files
    # if files are mat files
    if len(files) != 0 and files[0].endswith('.mat'):
      mat_files = files
    
    # if both mat list and image list not empty
    if len(img_files) != 0 and len(mat_files) != 0:
      # sort lists
      img_files.sort()
      mat_files.sort()

      # get the root without last directory
      root = root.replace('ground_truth', '')
      root = root.replace('images', '')
      print(root)
      print(len(img_files))
      print(img_files)
      print(len(mat_files))
      print(mat_files)

      mirror_images = []
      mirror_mat = []

      # if there is mirror function
      if mirror_func != None:
        for i in range(len(img_files)):
          mirror_func(root, img_files[i], mat_files[i])
          mirror_images.append(img_files[i].replace('.', '_mirror.'))
          mirror_mat.append(mat_files[i].replace('.', '_mirror.'))

        img_files += mirror_images
        mat_files += mirror_mat

      # if there is a crop function
      if crop_func != None:  
        done = False
        c = n_crop
        for i in range(len(img_files)):
          while done == False or c > 0:
            done = crop_func(root, img_files[i], mat_files[i], c)
            if done:
              c-=1
          c = n_crop 
      img_files = []
      mat_files = []


#6. Run the augmentation function

In [ ]:
root = 'CVHajjDataSet/'

data_augmentation(root, crop_func= crop_img, n_crop = 9, mirror_func= mirror_img)